# Comparison of LFP from single synaptic input to different cell models

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import LFPy
import neuron
from neuron import h
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots

np.random.seed(12345)

tstop = 8
dt = 2**-6

xmin, xmax = [-50, 70]
zmin, zmax = [-50, 130]

dx = 20
dz = 20
x_grid, z_grid = np.mgrid[xmin:xmax+dx:dx, zmin:zmax+dz:dz]
num_elecs = len(x_grid.flatten())
elec_grid_params = dict(
            sigma = 0.3,
            x = x_grid.flatten(),
            y = np.zeros(num_elecs),
            z = z_grid.flatten(),
            method = 'linesource',
        )


In [ ]:
def insert_synaptic_input(cell):

    synapse_parameters = dict(
                          idx = 0,
                          e = 0., # reversal potential
                          weight = 0.01, # synapse weight
                          record_current = True, # record synapse current
                          syntype = 'Exp2Syn',
                          tau1 = 0.1, #Time constant, rise
                          tau2 = 1.0, #Time constant, decay
                          )
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([1.]))
    return synapse, cell


def plot_results(cell, figname, figtitle, subplot_marker):

    elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
    M_elec = elec.get_transformation_matrix()
    eaps = M_elec @ cell.imem * 1000

    xmin = np.min(elec_grid_params["x"])
    xmax = np.max(elec_grid_params["x"])
    zmin = np.min(elec_grid_params["z"])
    zmax = np.max(elec_grid_params["z"])

    eap_idxs = np.where((np.abs(elec_grid_params["z"] + 10) < 1e-9) &
                        (elec_grid_params["x"] > 0))[0]

    eap_clrs = {idx: plt.cm.Reds_r(num / (len(eap_idxs)))
                for num, idx in enumerate(eap_idxs)}

    fig = plt.figure(figsize=[2, 4.])

    fig.suptitle(figtitle)
    ax_morph = fig.add_axes([0.05, 0.22, 0.9, 0.68], frameon=False, aspect=1,
                            xticks=[], yticks=[], xlim=[xmin - 5, xmax + 10],
                            ylim=[zmin - 10, zmax + 5])

    ax_eap = fig.add_axes([0.1, 0.01, 0.85, 0.17], 
                          frameon=False, xticks=[], yticks=[],
                          ylim=[-1.05, 0.5])

    for n, elec_idx in enumerate(eap_idxs[::-1]):
        c = eap_clrs[elec_idx]
        eap_norm = eaps[elec_idx] / np.max(np.abs(eaps[elec_idx]))
        ls = '-'# if n == (len(eap_idxs) - 1) else '-'
        ax_eap.plot(cell.tvec, eap_norm, c=c, lw=1, ls=ls)
        x = int(elec_grid_params["x"][elec_idx])
        ax_eap.text(6., -1.05 + n * 0.25, "x={:d} µm".format(x), c=c)

    zips = []
    for x, z in cell.get_pt3d_polygons():
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips, edgecolors='none',
                             facecolors='0.8', zorder=-1, 
                             rasterized=False)
    ax_morph.add_collection(polycol)

    dz = np.abs(np.diff(elec.z))[0]
    num_elecs = len(elec.x)
    eap_norm = dz * 0.9 / np.max(np.abs(eaps))
    t_norm = cell.tvec / cell.tvec[-1] * dz * 0.7
    for elec_idx in range(num_elecs):
        c = eap_clrs[elec_idx] if elec_idx in eap_idxs else 'k'
        x, z = elec.x[elec_idx], elec.z[elec_idx]
        ax_morph.plot(x, z, '.', c='k', ms=3)
        eap = eaps[elec_idx] * eap_norm
        ax_morph.plot(x + t_norm, z + eap, c=c, lw=1)

    ax_morph.plot([20, 40], [15, 15], c='gray', lw=1)
    ax_morph.text(30, 17, "20 µm", ha="center", c='gray')

    ax_morph.plot([82, 82], [-10 - 10 * eap_norm, -10], c='k', lw=1,
                  clip_on=False)
    ax_morph.text(79, -20, "10 µV", ha="right",
                  c='k', va="center")

    mark_subplots(ax_morph, subplot_marker, xpos=-0.01, ypos=1.1)

    ax_eap.text(-1, 0.07, "normalized\nLFP", fontsize=10)
    ax_eap.plot([2, 3], [-0.05, -0.05], c='k', lw=1)
    ax_eap.text(2.5, - 0.02, "1 ms", va="bottom", ha='center')

    fig.savefig("{}.pdf".format(figname))
    cell.__del__()


In [ ]:
cell = ns.return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
syn, cell = insert_synaptic_input(cell)
h.dt = dt

for sec in neuron.h.allsec():
    if "soma" in sec.name():
        print("g_pas: {}, e_pas: {}, cm: {}, "
              "Ra: {}, soma_diam: {}, soma_L: {}".format(sec.g_pas,
                                                         sec.e_pas, sec.cm,
                                                         sec.Ra, sec.diam,
                                                         sec.L))

cell.simulate(rec_imem=True, rec_vmem=True)

plot_results(cell, "fig_hay_passive", "reconstructed neuron", "A")
cell.__del__()

## Do the same for ball and stick neuron

In [ ]:
cell = ns.return_ball_and_stick_cell(tstop, dt)
syn, cell = insert_synaptic_input(cell)
for sec in neuron.h.allsec():
    # Insert same passive params as Hay model
    sec.g_pas = 3.3333333333333335e-05#3.38e-05
    sec.e_pas = -70.0#-90
    sec.cm = 1.0
    sec.Ra = 150
h.dt = dt

cell.simulate(rec_imem=True, rec_vmem=True)
plot_results(cell, "fig_ball_and_stick", "ball-and-stick", "B")
cell.__del__()

## Do the same for two-compartment neuron model

In [ ]:
cell = ns.return_two_comp_cell(tstop, dt)
syn, cell = insert_synaptic_input(cell)
h.dt = dt
for sec in neuron.h.allsec():
    # Insert same passive params as Hay model
    sec.g_pas = 3.3333333333333335e-05#3.38e-05
    sec.e_pas = -70.0 #-90
    sec.cm = 1.0
    sec.Ra = 150

cell.simulate(rec_imem=True, rec_vmem=True)

plot_results(cell, "fig_two_comp", "two-compartment", "C")
cell.__del__()